In [9]:
# %load ../../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn import neighbors, metrics, model_selection
import sklearn.tree as tree

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)
 
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

Numericas:
- 'Administrative': Numero de paginas de tipo administrativo
- 'Administrative_Duration': mins, duracion en este tipo de paginas
- 'Informational': Numeor de paginas de tipo infomracion
- 'Informational_Duration': mins, duracion en este tipo de paginas
- 'ProductRelated': Numeor de paginas de productos relacionados vistos
- 'ProductRelated_Duration': mins, timepo que ha estado en las paginas de productos relacionados
- 'BounceRates': Porcentege de visitante que entran y salen sin interacutar
- 'ExitRates': Porcentaje de persona que slieron despues de interacutar
- 'PageValues': Media de la valoracion de la pagina antes de hacer una transaccion comercial
- 'SpecialDay': (0,1) Si el dia esta cerca a un dia especial
- 'OperatingSystems':
- 'Browser':
- 'Region':
- 'TrafficType':

Categoricas:
- 'Month': 'Feb': 2,'Mar': 3,'May': 5,'June': 6,'Jul': 7,'Aug': 8,'Sep': 9,'Oct': 10,'Nov': 11,'Dec': 12
- 'Weekend': 1(Fasle), 2(True)
- 'VisitorType': 1(Ha vuelto), 2(Nuevo), 3(otro)

Clases 'Revenue'

In [6]:
datos = pd.read_csv('online_shoppers_intention.csv')

datos_ok = datos.copy()
categorico = ['Month','Weekend','VisitorType','Revenue']

for i,val in enumerate(datos_ok.columns):
    if val == 'Month':
        mesesDic = dict(zip(['Feb', 'Mar', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], [2,3,5,6,7,8,9,10,11,12]))
        datos_ok[val].replace(mesesDic, inplace=True)
    elif val in categorico:
        labels = datos_ok[val].astype('category').cat.categories.tolist()
        dict_make = {val : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
        datos_ok[val].replace(dict_make[val], inplace=True)

X_all = np.array(datos_ok.drop(columns = 'Revenue').copy())
y_all = np.array(pd.factorize(datos_ok['Revenue'])[0])

## Elimnacion de valores Perididos

In [7]:
n = np.isnan(X_all)
n = np.sum(n,axis=1).reshape(-1,1).astype('bool')
n = n == False
X = X_all[n.flatten()]
y = y_all[n.flatten()]

## One Hot Ecnode busqueda

In [38]:
#hanwei
print(X_all.shape)
n = np.isnan(X_all)

# Se fija la semilla de numpy para que la generación aleatoria siempre nos de los mismos números
np.random.seed(12)

# Lllamada a la función train_test_split y guardado del resultado
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

# Realizamos el proceso para KNN por lo que hay que llamar al constructor de dicho clasificador
clf = tree.DecisionTreeClassifier()

# Llamada a la función que realiza el aprendizaje del clasificador
clf.fit(X_train,y_train)

# Llamada a la función que realiza la predicción de los datos de entrenamiento
predTrain = clf.predict(X_train)

# Llamada a la función que calcula el porcentaje de acierto para los datos de entrenamiento
accTrain = round(metrics.accuracy_score(y_train,predTrain)*100,2)
print('El rendimiento en entrenamiento con todas las variables es el {}%'.format(accTrain))

# Llamada a la función que realiza la predicción de los datos de test
predTest = clf.predict(X_test)

# Llamada a la función que calcula el porcentaje de acierto para los datos de test
accTest = round(metrics.accuracy_score(y_test,predTest)*100,2)
print('El rendimiento en test con todas las variables es el {}%'.format(accTest))

(12330, 17)
El rendimiento en entrenamiento con todas las variables es el 100.0%
El rendimiento en test con todas las variables es el 85.9%


In [23]:
num = np.sum(datos['Revenue'] == 1)
num2 = np.sum(datos['Revenue'] == 0)
print(num,num2)
print(y_all)

x_pos = X_all[y_all == 1]
x_pos = X_all[y_all == 0]

1908 10422
[0 0 0 ... 0 0 0]


<br><br><br><br><br><br>

In [18]:
XsinOutliers = X.copy()
YsinOutliers = y.copy()
# Se importa la función que permite seleccionar variables a partir de un modelo
from sklearn.feature_selection import SelectFromModel

np.random.seed(12)
semilla = 123

# Se llama al constructor del árbol de decisión
arbolDecision = tree.DecisionTreeClassifier()

# Se aplica la selección a partir del modelo
model = SelectFromModel(estimator=arbolDecision)

# Se entrena el objeto de la clase SelectFromModel
model.fit(XsinOutliers, YsinOutliers)

# Se transforman los datos originales de entrenamiento de acuerdo a las variables seleccionadas
X_new = model.transform(XsinOutliers)

# Se obtienen las variables seleccionadas: lista de booleanos
booleanas =  model.get_support()

# Se obtienen los nombres de las variables seleccionadas
variablesSeleccionadas = [datos_ok.columns[:-1][i] for i in range(len(datos_ok.columns)-1) if booleanas[i]==True]
print("Las variables seleccionadas son ", variablesSeleccionadas)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_new, YsinOutliers, train_size=0.7)
# Llamada al constructor del árbol de decisión
arbolDecision = tree.DecisionTreeClassifier()

# Entrenamiento del árbol de decisión con los datos transformados
arbolDecision.fit(X_train,y_train)

# Predicción de las clases de los datos de entrenamiento transformados
prediccionesTrain = arbolDecision.predict(X_train)
prediccionesTest = arbolDecision.predict(X_test)

# Se obtiene el accuracy en enrenamiento del nuevo árbol
accuracyTrain =  metrics.accuracy_score(y_train,prediccionesTrain)*100
print(accuracyTrain)

# Obtenición del rendimiento con los datos de test
accuracyTest =  metrics.accuracy_score(y_test,prediccionesTest)*100
print(accuracyTest)



Las variables seleccionadas son  ['Administrative_Duration', 'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues']
99.9652012527549
84.92557510148849


In [22]:
from scipy import stats
def leaveOneOut(clasificador, X, y):
    """
    :param clasificador: Instancia de un clasificador de Scikit-Learn entrenada (con fit hecho con los datos de train o el subconjunto seleccionado)
    :param X: Matriz con los ejemplos de entrenamiento completo (para hacer leave one out)
    :param y: Vector con la salida de los ejemplos de entrenamiento completo (correspondientes a X)
    :return: Vector con la salida obtenida para cada ejemplo de X (siguiendo el esquema leave-one-out)
    """
    # En caso de que el clasificador no sea KNN simplemente se utiliza el clasificador aprendido y se predicen los ejemplos con dicho clasificador
    if type(clasificador) != neighbors.KNeighborsClassifier:
        # Utiliza el método predict para obtener la predicción
        prediction = clasificador.predict(X)
        return prediction
    
    # En otro caso será una instancia de KNeighborsClassifier
    # kneighbors nos devuelve los vecinos más cercanos de cada ejemplo en X y las distancias asociadas
        # distancias y vecinos serán matrices de numEjemplos x k+1 elementos
    distancias, vecinos = clasificador.kneighbors(X, n_neighbors=clasificador.n_neighbors + 1, return_distance=True)
    # Realmente no estamos interesados en el propio vecino, sino en la clase a la que pertenece
    # Transformamos los vecinos a una matriz con la clase de cada vecino
    # Accedemos a las clases de los ejemplos de entrenamiento (atributo -y del clasificador knn) 
        # y cogemos los elementos en las posiciones de los vecinos calculados anteriormente
    vecinosClase = clasificador._y[vecinos] # probar si funciona igual porque entiendo que si y es mucho más fácil (para train va mal)
    
    # Ahora queda decidir la clase de salida para cada uno de los ejemplos
    # Debemos tener en cuenta: Si la distancia al primer vecino es 0, el vecino es el propio ejemplo y por tanto no lo usamos para decidir
    #   1. Obtenemos una máscara con los ejemplos en los que ocurre esto
    #   2. Sustituimos la clase del primer vecino por la del último (el k+1 que sino no se utiliza)
    #   3. Obtenemos la moda (stats.mode) (el valor más repetido de clase) para cada fila (cogiendo los k primeros valores)
    mascara = distancias[:,0] == 0
    vecinosClase[mascara,0] = vecinosClase[mascara,-1]
    prediction =  stats.mode(vecinosClase[:,:-1],axis=1)[0]

    return prediction

    """
    Esta función se encajar de ejecutar un método de selección de instancias y dar los resultados sobre train y test
    con el clasificador introducido como parámetro
    :param funcionMetodo: Función con el método de selección de instancias a ejecutar
    :param clasificador: Instancia del clasificador a utilizar para obtener la precisión en train y test
    :param train: Conjunto de datos de entrenamiento leído del formato keel con campos data y target
    :param test: Conjunto de datos de test leído del formato keel con campos data y target
    :return: tupla con la máscara de ejemplos seleccionados, la precisión en train, precisión en test y porcentaje de reducción obtenido
    """

In [28]:
def ejecutaMetodoIS(funcionMetodo, clasificador, X_train, y_train, X_test, y_test, verbose=True):
    # Obtenemos el nombre del método. Como es una función utilizamos su campo __name__
    nombreMetodo = funcionMetodo.__name__
    if verbose:
        print ("Ejecutando " + nombreMetodo + "...")
    # Ejecutamos el método de selección de instancias correspondiente con los ejemplos de entrenamiento y sus clases
        # Utilizad todos los métodos con los parámetros por defecto y almacenad la máscara devuelta en una variable llamada S
    S = funcionMetodo(X_train,y_train)
    
    # Entrenamos el clasificador correspondiente solo con los ejemplos seleccionados (aplicamos la máscara S)
    clasificador.fit(X_train[S,:],y_train[S])

#     # Obtenemos las salidas para train mediante leaveOneOut
    predictionTrain = leaveOneOut(clasificador,X_train,y_train)

#     # Obtenemos la precisión en train con las salidas obtenidas (tanto por uno)
    accTrain = metrics.accuracy_score(predictionTrain,y_train)

#     # Obtenemos las salidas para test 
#     # Obtenemos la precisión en test  (tanto por uno)
    accTest = clasificador.score(X_test,y_test)

    # Calculamos el porcentaje de reducción
    reduction = 100 - S.sum() / float(S.size)*100

    if verbose:
        # print("Dataset " + dataset)
        print("Resultados " + nombreMetodo + " python")
        print("Precisión en train: {}".format(accTrain))
        print("Precisión en test: {}".format(accTest))
        print("Reducción " + nombreMetodo + ": {} de {}".format(S.sum(), S.size))
        print("Reducción: %2.2f%%" % reduction)
        # En caso de ser un árbol, imprimimos el número de reglas obtenidas
        if type(clasificador) == tree.DecisionTreeClassifier:
            print("Número de reglas: {}".format(clasificador.tree_.node_count))

    # Mostramos los resultados obtenidos por el método gráficamente
    # mostrar(clasificador, X_train, y_train, X_test, y_test,  nombreMetodo, X_train[S, :], y_train[S])

    # Si es un árbol, devolvemos el número de reglas obtenidas
    nReglas = -1
    if type(clasificador) == tree.DecisionTreeClassifier:
        nReglas = clasificador.tree_.node_count
    return S, accTrain, accTest, reduction, nReglas

In [24]:
def CNN(X, y, k=1):
    """
    Algoritmo CNN para la selección de instancias. Se comienza con dos ejemplos aleatorios (uno de cada clase) y cada ejemplo
    que se falla al ser clasificado por los ya seleccionados se añade a la selección (se para cuando ya no se añaden más ejemplos)
    :param X: Matriz con los ejemplos de entrenamiento (se asume que los ejemplos están normalizados)
    :param y: Vector con la salida de los ejemplos en X
    :param k: Valor de k a utilizar en ENN
    :return: Vector con la máscara de instancias seleccionadas 
            (La posición S[i]=True indica que la instancia i ha sido seleccionada y False lo contrario)
    """
    # Como usamos aleatorios en RNN establecemos la semilla para que el test sea correcto y siempre obtengamos el mismo resultado
    np.random.seed(12312)
    # Creamos el clasificador knn con el valor de k dado
    knn = neighbors.KNeighborsClassifier(n_neighbors=k)

    # S es una máscara de booleanos que indica si el ejemplo en la posición i está seleccionado (True) o no lo está (False)
    S = np.zeros((X.shape[0]),dtype=bool)

    # Inicialmente, seleccionamos aleatoriamente una instancia de cada clase
    # Anotamos el número de clases
    nClases = np.unique(y)

    # Para cada clase, buscamos los índices de los ejemplos de dicha clase y seleccionamos uno de ellos aleatoriamente
    # Dicho ejemplo coge el valor True en S
    for c in nClases:
        indicesClase = np.where(y == c)[0]  # np.where devuelve una tupla, nos interesa la primera parte que contiene los índices de las posiciones que no son 0 o False
        # Elegid una instancia aleatoriamente de las de la clase: utilizad randint de Numpy para ello
        instanciaAleatoria = indicesClase[np.random.randint(0,indicesClase.size)]
        # Modificad la máscara S de acuerdo a la instancia seleccionada
        S[instanciaAleatoria] = True

    # Solo evaluamos los ejemplos que no están en S (los que están se aciertan por definición al considerlos
    # como vecinos de sí mismos). Estos ejemplos son los únicos que podemos añadir a S
    notS = S == False

    # "Entrenamos" knn con los ejemplos seleccionados
    knn.fit(X[S,:],y[S])
    
    # Inicializamos el número de fallados para entrar en el bucle. El algoritmo termina cuando no se fallan ejemplos
    fallados = -1
    while fallados != 0:
        # Ponemos el contador de fallados a 0 y comprobamos todos los ejemplos no seleccionados
        fallados = 0
        # Obtenemos los índices de los ejemplos no seleccionados: utilizad np.where como se ha hecho anteriormente
        indices = np.where(notS)[0]

        # Para cada ejemplo no seleccionado (y en orden aleatorio) comprobamos si dicho ejemplo se falla         
        # con las instancias en S actualmente. Si se falla, el ejemplo se añade a S, se reentrena KNN y se suma uno a fallados
        for i in np.random.permutation(indices):
            if y[i] != knn.predict(X[i, :].reshape(1, -1)):
                S[i] = True
                knn = knn.fit(X[S, :], y[S])
                fallados += 1

        # Recalculamos los ejemplos a estudiar en la próxima iteración, aquellos no seleccionados
        notS = np.logical_not(S)
        print("CNN, fin de iteración, fallados: {}, ejemplos en S: {}".format(fallados, np.sum(S)))

    return S

In [25]:
def RNN(X, y, k=1):
    """
    Algoritmo RNN para la selección de instancias. Se parte de la selección obtenida con CNN y se eliminan aquellas instancias
    que no provoquen que se falle ninguna instancia no seleccionada (las seleccionadas se aciertan por definición al estar en el subconjunto)
    :param X: Matriz con los ejemplos de entrenamiento (se asume que los ejemplos están normalizados)
    :param y: Vector con la salida de los ejemplos en X
    :param k: Valor de k a utilizar en ENN
    :return: Vector con la máscara de instancias seleccionadas 
            (La posición S[i]=True indica que la instancia i ha sido seleccionada y False lo contrario)
    """
    # Como usamos aleatorios en RNN establecemos la semilla para que el test sea correcto y siempre obtengamos el mismo resultado
    np.random.seed(12312)
    # Creamos el clasificador knn con el valor de k dado
    knn = neighbors.KNeighborsClassifier(n_neighbors=k)

    # En RNN partimos de la selección realizada por CNN y eliminamos ejemplos de la misma
    # S es una máscara de booleanos que indica si el ejemplo en la posición i está seleccionado (True) o no lo está (False)
    S = CNN(X,y,k)

    # Entrenamos KNN con el conjunto de instancias seleccionadas por CNN
    knn.fit(X[S,:],y[S])

    # Obtenemos la máscara de instancias no seleccionadas
    # No podemos fallar ninguna de estas si queremos eliminar una instancia de S
    notS = S == False

    # Recorremos las instancias seleccionadas para ver si podemos eliminarlas o no
    # Obtenemos para ello los índices de las instancias: utilizad np.where
    indices = np.where(S)[0]

    # y las recorremos aleatoriamente:
    for i in np.random.permutation(indices):
        # Asumimos que podemos eliminar la instancia y la "eliminamos" temporalmente: intercambio de valores en las máscaras S y Snot
        S[i],notS[i] = notS[i],S[i]

        # Entrenamos knn una vez eliminada la instancia
        knn.fit(X[S,:],y[S])

        # Obtenemos las salidas para los ejemplos que no están seleccionados
        salidas = knn.predict(X[notS,:])

        # Si todos los ejemplos no seleccionados son acertados (no se falla ninguno) mantenemos la eliminación (no hacemos nada)
            # En otro caso, si fallamos alguna instancia, debemos restablecer la instancia eliminada: deshacer el intercambio en las máscaras
        if np.sum(y[notS] != salidas) != 0:
            S[i], notS[i] = notS[i], S[i]

    return S

In [32]:
def calcularMediaDesviacion(datos):
    media = datos.mean(axis=0)
    desviacion = datos.std(axis=0)
    return media,desviacion

In [33]:
def deteccionOutliers(datos,media,desviacion,k=2):
    variablesOutliers = (datos < (media - k*desviacion)) | (datos > (media + k*desviacion))
    mascaraOutliers = variablesOutliers.sum(axis=1,dtype=bool)
    indicesOutliers = np.array(range(len(datos)))[mascaraOutliers]
    return indicesOutliers

In [37]:
index_col_num = np.ones(len(datos_ok.columns))

for i,val in enumerate(datos_ok.columns):

    if val in categorico:
        index_col_num[i] = 0

## no cogemos la ultima variable porque es la clase
index_col_num = index_col_num[:-1].astype('bool')

media,desviacion = calcularMediaDesviacion(X.T[index_col_num].T)

indicesOutliers = deteccionOutliers(X.T[index_col_num].T,media,desviacion)
print('existe',len(indicesOutliers), 'filas de Outliers.')

XsinOut = np.delete(X, indicesOutliers, axis=0)
YsinOut = np.delete(y, indicesOutliers, axis=0)

existe 5715 filas de Outliers.


In [38]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(XsinOut, YsinOut, train_size=0.7)

In [41]:
%%time
knn = neighbors.KNeighborsClassifier()
_, accTrain, accTest, reduction, _ = ejecutaMetodoIS(RNN, knn, X_train, y_train, X_test, y_test)

Ejecutando RNN...
CNN, fin de iteración, fallados: 1108, ejemplos en S: 1110
CNN, fin de iteración, fallados: 312, ejemplos en S: 1422
CNN, fin de iteración, fallados: 28, ejemplos en S: 1450
CNN, fin de iteración, fallados: 1, ejemplos en S: 1451
CNN, fin de iteración, fallados: 0, ejemplos en S: 1451
Resultados RNN python
Precisión en train: 0.8415584415584415
Precisión en test: 0.8490661282180717
Reducción RNN: 1210 de 4620
Reducción: 73.81%
Wall time: 4min 2s
